<a href="https://colab.research.google.com/github/TokyoYoshida/GoogleColab/blob/master/playground_deep_learning_keiba2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [160]:
%cd /content/drive/'My Drive'/'★個人専用ファイル'/'開発用・個人サービス開発'/'GoogleColab'/

/content/drive/My Drive/★個人専用ファイル/開発用・個人サービス開発/GoogleColab


In [161]:
%ls


data/
duel_dqn_Pendulum-v0_weights.h5f
h5f/
model.txt
playground_deep_learning_keiba
playground_dqn_keiba
playground_dqn_keiba_try_layer_add_depth
playground_dqn_keiba_try_long_term_reward
playground_dqn_keiba_try_simple
playground_dqn_keiba_try_simple2
playground_dqn_keiba_try_simple_x_1
playground_dqn_study.ipynb
playground_dqn_study_my_try.ipynb
playground_dqn_study_my_try_modify_reward
playground.ipynb


In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from pprint import pprint

In [165]:
!pip install tensorflow==2.2.0

In [0]:
!pip install -q keras

In [0]:
import keras
from keras.utils import np_utils


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LeakyReLU
from keras.optimizers import RMSprop


In [0]:
from PIL import Image
from matplotlib.pyplot import imshow

#img = Image.open("test.jpg")
#imshow(img)

In [0]:
pd.set_option('display.max_rows', 500)

In [0]:
# 競馬データの作成

In [0]:
ldat = pd.read_pickle("./data/keiba_dat_res_20190611_sorted.pkl")

In [173]:
28737.800000000003/ 37564

0.765035672452348

In [0]:
# 前回の3位以内にマークを付ける
ldat["prev_high_rank"] = 0
ldat.loc[ldat["prev_rank"] <= 3, "prev_high_rank"] = 1

In [0]:
# 3位以内にマークを付ける
ldat["high_rank"] = 0
ldat.loc[ldat["rank"] <= 3, "high_rank"] = 1

In [0]:
# 前回ランクを段階化
ldat["prev_rank_level"] = round(ldat["prev_rank"]  / 4)

In [0]:
# ランクを段階化
ldat["rank_level"] = round(ldat["rank"]  / 4)

In [0]:
# 前回人気を段階化
ldat["prev_popularity_level"] = round(ldat["prev_popularity"] / 4)

In [0]:
# 前回人気を段階化
ldat["popularity_level"] = round(ldat["popularity"] / 4)

In [0]:
# 前回オッズを段階化
ldat["prev_odds_level"] = round(ldat["prev_odds"] / 4)

In [0]:
# 前回人気を段階化
ldat["opdds_level"] = round(ldat["odds"] / 4)

In [0]:
# 前回スピードを出す
ldat["prev_speed"] = ldat["prev_distance"] / ldat["prev_time_float"] 

In [0]:
# 前回スピードを段階化
ldat["prev_speed_level"] =  round(ldat["prev_speed"]*10000000)

In [0]:
# ここから競馬分析　start

In [0]:
corr = ldat.corr()

In [186]:
corr["rank"].sort_values(ascending=False)

rank                     1.000000
rank_level               0.967962
popularity               0.589481
popularity_level         0.572177
odds                     0.466596
opdds_level              0.466482
prev_rank                0.379749
prev_rank_level          0.365739
prev_popularity          0.363686
prev_popularity_level    0.352791
prev_odds                0.307900
prev_odds_level          0.307832
diff_arrival             0.221195
horse_num                0.089638
jockey                   0.084112
prev_jockey              0.061807
prev_diff_arrival        0.058011
age                      0.037860
direction                0.037598
trainer                  0.020521
prev_age                 0.020142
prev_horse_num           0.017106
prev_trainer             0.015965
prev_grade               0.013780
prev_month               0.010950
prev_direction           0.010648
frame_num                0.010617
year                     0.008203
name                     0.004263
day           

In [0]:
# ここから競馬分析　end

In [0]:
sdat = ldat.sample(frac=1).reset_index(drop=True).dropna(0)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする start

In [0]:
winner = sdat.query("prev_high_rank == 1")

In [0]:
loser = sdat.query("prev_high_rank != 1").head(winner.count()[0])

In [0]:
catted = pd.concat([winner,loser])

In [0]:
sampled = catted.sample(frac=1)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする end

In [0]:
# x , yデータを作る start

In [0]:
x = sampled[['frame_num', 'horse_num', 'name', 'sex', 'age', 'amount',
       'jockey', 'odds', 'popularity', 'weight',
       'diff_weight', 'trainer', 'race_num', 'grade', 'ground', 'direction',
        'distance', 'weather', 'condition', 'start_time_float', 'year',
       'month', 'day', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
       'prev_name', 'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
       'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
       'prev_weight', 'prev_diff_weight', 'prev_trainer', 'prev_race_num',
       'prev_grade', 'prev_ground', 'prev_direction',
       'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float',
       'prev_year', 'prev_month', 'prev_day']].dropna(0)

In [0]:
# x = sdat[['frame_num', 'horse_num', 'sex', 'age', 'amount',
#        'odds', 'popularity', 'weight',
#        'diff_weight', 'trainer', 'grade', 'ground', 'direction',
#         'distance', 'weather', 'condition', 'start_time_float', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
#        'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
#        'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
#        'prev_weight', 'prev_diff_weight', 'prev_trainer',
#        'prev_grade', 'prev_ground', 'prev_direction',
#        'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float'
#        ]].dropna(0)

In [0]:
# x = sdat[["odds",'prev_rank','prev_diff_arrival']].dropna(0)

In [0]:
cols = ["popularity_level","opdds_level","prev_rank_level","prev_popularity_level","prev_odds_level","horse_num","jockey","prev_jockey"]

In [0]:
x = sampled[cols].dropna(0)

# lastx_tmpは最終評価用に使用、勝ち負けを半々にしていない
lastx_tmp = sdat[cols].dropna(0)

In [0]:
#x = sampled[['high_prev_popularity']].dropna(0)

In [0]:
#y = sampled[["high_rank"]].dropna(0)

In [0]:
y_np = np_utils.to_categorical(sampled[["high_rank"]].dropna(0))
y = pd.DataFrame(y_np,columns=["lose","win"],index=sampled.index)

# lasty_tmpは最終評価用に使用、勝ち負けを半々にしていない
lasty_np = np_utils.to_categorical(sdat[["high_rank"]].dropna(0))
lasty_tmp = pd.DataFrame(lasty_np,columns=["lose","win"],index=sdat.index)

In [204]:
len(lasty)

352595

In [205]:
sampled["high_rank"]

32603     0
143905    0
5170      0
39244     0
78354     0
         ..
119137    0
78353     1
141427    1
27283     1
125113    0
Name: high_rank, Length: 208460, dtype: int64

In [0]:
# x , yデータを作る end

In [0]:
# 競馬データの作成 終わり

In [0]:
# 学習、テストデータの作成

In [0]:
trainlen = int(len(x)/2)

In [0]:
trainx = x.head(trainlen)
trainy = y.head(trainlen)

In [0]:
testx = x.tail(len(x)-trainlen)
testy = y.tail(len(x)-trainlen)

In [0]:
testorg = sampled.tail(len(x)-trainlen) 

In [0]:
# 学習、テストデータの作成　終わり

In [0]:
# 最終検証用データからトレーニングデータを取り除く
lastx = lastx_tmp[~lastx_tmp.index.isin(trainx.index)]
lasty = lasty_tmp[~lasty_tmp.index.isin(trainy.index)]
lastsdat = sdat[~sdat.index.isin(trainx.index)]

In [215]:
lastsdat

,rank,frame_num,horse_num,name,sex,age,amount,jockey,time,diff_arrival,odds,popularity,weight,diff_weight,trainer,race_num,grade,ground,direction,unknown,distance,weather,condition,start_time,year,month,day,prev_rank,prev_frame_num,prev_horse_num,prev_name,prev_sex,prev_age,prev_amount,prev_jockey,prev_time,prev_diff_arrival,prev_odds,prev_popularity,prev_weight,prev_diff_weight,prev_trainer,prev_race_num,prev_grade,prev_ground,prev_direction,prev_unknown,prev_distance,prev_weather,prev_condition,prev_start_time,prev_year,prev_month,prev_day,winner,return,prev_time_float,start_time_float,prev_start_time_float,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,popularity_level,prev_odds_level,opdds_level,prev_speed,prev_speed_level
1,17.0,1,2,1376,1,8,57.0,304,1:22.9,9,148.4,17.0,492.0,0.0,115,12,9,0,2,-,1400.0,3,0,16:15,2008,4,26,12.0,4.0,8.0,1376.0,1.0,8.0,57.0,57.0,1:12.6,9.0,89.0,14.0,492.0,-2.0,115.0,8.0,9.0,1.0,0.0,-,1200.0,0.0,0.0,13:40,2008.0,1.0,5.0,False,0.0,1561165956,1561220100,1561210800,0,0,3.0,4.0,4.0,4.0,22.0,37.0,7.686563e-07,8.0
4,6.0,2,3,21581,0,5,55.0,68,1:10.4,4,73.0,11.0,466.0,2.0,164,6,9,0,0,-,1200.0,0,0,12:30,2012,4,8,16.0,3.0,5.0,21581.0,0.0,5.0,55.0,334.0,1:40.1,7.0,66.3,13.0,464.0,-1.0,164.0,7.0,9.0,0.0,0.0,外,1600.0,3.0,3.0,13:20,2012.0,1.0,22.0,False,0.0,1561167606,1561206600,1561209600,0,0,4.0,2.0,3.0,3.0,17.0,18.0,1.024874e-06,10.0
6,12.0,5,7,24470,1,3,56.0,118,2:02.4,3,433.2,13.0,494.0,12.0,55,12,9,0,0,-,2000.0,0,0,16:10,2011,12,17,12.0,8.0,14.0,24470.0,1.0,3.0,55.0,26.0,2:30.7,9.0,202.8,13.0,482.0,-2.0,55.0,6.0,9.0,0.0,2.0,-,2400.0,3.0,0.0,12:30,2011.0,11.0,19.0,False,0.0,1561170642,1561219800,1561206600,0,0,3.0,3.0,3.0,3.0,51.0,108.0,1.537308e-06,15.0
8,14.0,6,11,11205,1,3,56.0,63,2:28.8,7,30.2,9.0,504.0,-6.0,69,11,2,0,2,-,2400.0,1,1,15:40,2008,5,3,4.0,8.0,9.0,11205.0,1.0,3.0,56.0,20.0,2:14.6,6.0,13.2,6.0,510.0,0.0,69.0,9.0,9.0,0.0,0.0,外,2200.0,0.0,0.0,14:30,2008.0,4.0,5.0,False,0.0,1561169676,1561218000,1561213800,0,0,1.0,4.0,2.0,2.0,3.0,8.0,1.409200e-06,14.0
14,1.0,1,1,9921,0,4,55.0,64,1:12.2,0,5.7,3.0,450.0,-4.0,39,12,9,1,0,-,1200.0,0,0,16:20,2009,3,28,8.0,2.0,4.0,9921.0,0.0,4.0,55.0,49.0,1:25.1,1.0,4.4,2.0,454.0,-4.0,39.0,8.0,9.0,1.0,0.0,-,1400.0,1.0,1.0,13:40,2009.0,2.0,22.0,True,5.7,1561166706,1561220400,1561210800,0,1,2.0,0.0,0.0,1.0,1.0,1.0,8.967652e-07,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542846,11.0,8,15,13001,0,4,55.0,184,1:10.0,2,213.1,15.0,436.0,-1.0,453,8,9,1,0,-,1150.0,1,1,14:01,2009,7,5,15.0,3.0,6.0,13001.0,0.0,3.0,51.0,249.0,1:49.6,17.0,23.7,8.0,430.0,14.0,24.0,1.0,9.0,1.0,2.0,-,1700.0,0.0,1.0,10:10,2008.0,6.0,7.0,False,0.0,1561168176,1561212060,1561198200,0,0,4.0,3.0,2.0,4.0,6.0,53.0,1.088928e-06,11.0
542848,4.0,8,15,35258,1,4,56.0,57,1:52.0,7,9.5,7.0,512.0,4.0,115,11,9,1,0,-,1800.0,0,2,15:35,2014,1,11,4.0,3.0,6.0,35258.0,1.0,3.0,56.0,83.0,1:53.5,9.0,7.2,4.0,508.0,-4.0,115.0,9.0,9.0,1.0,0.0,-,1800.0,0.0,2.0,14:35,2013.0,12.0,22.0,False,0.0,1561168410,1561217700,1561214100,0,0,1.0,1.0,1.0,2.0,2.0,2.0,1.152983e-06,12.0
542849,3.0,7,13,13702,0,3,54.0,75,1:50.3,1,4.6,2.0,458.0,-4.0,132,2,9,0,0,-,1800.0,0,0,10:20,2009,2,7,3.0,5.0,10.0,13702.0,0.0,3.0,54.0,75.0,1:52.3,14.0,3.6,2.0,462.0,-6.0,132.0,4.0,9.0,0.0,2.0,-,1800.0,0.0,0.0,11:30,2009.0,1.0,24.0,False,0.0,1561168338,1561198800,1561203000,1,1,1.0,1.0,0.0,0.0,1.0,1.0,1.152983e-06,12.0
542853,3.0,7,10,25151,1,7,58.0,194,1:46.1,3,15.8,5.0,482.0,-4.0,143,11,9,0,0,外,1800.0,1,1,15:35,2015,5,16,12.0,3.0,6.0,25151.0,1.0,7.0,56.0,79.0,1:33.6,4.0,8.6,3.0,486.0,0.0,143.0,11.0,3.0,0.0,0.0,外,1600.0,2.0,0.0,15:45,2015.0,4.0,5.0,False,0.0,1561167216,1561217700,1561218300,0,1,3.0,1.0,1.0,1.0,2.0,4.0,1.024874e-06,10.0


In [216]:
lasty

,lose,win
1,1.0,0.0
4,1.0,0.0
6,1.0,0.0
8,1.0,0.0
14,0.0,1.0
...,...,...
542846,1.0,0.0
542848,1.0,0.0
542849,0.0,1.0
542853,0.0,1.0


In [0]:
#!pip uninstall -y keras

In [0]:
#!pip uninstall -y tensorflow

In [0]:
batch_size = 128
num_classes = 2
epochs = 5


In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x.shape[1],)))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(256, activation=None))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(128, activation=None))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2)) # 出力層2ノード,全結合
model.add(Activation("sigmoid"))

In [221]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               4608      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [223]:
history = model.fit(trainx, trainy,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(testx, testy))


Train on 104230 samples, validate on 104230 samples
Epoch 1/5
104230/104230 [==============================] - 8s 79us/step - loss: 0.6047 - accuracy: 0.7323 - val_loss: 0.4818 - val_accuracy: 0.7529
Epoch 2/5
104230/104230 [==============================] - 8s 79us/step - loss: 0.4849 - accuracy: 0.7547 - val_loss: 0.4771 - val_accuracy: 0.7580
Epoch 3/5
104230/104230 [==============================] - 8s 81us/step - loss: 0.4822 - accuracy: 0.7579 - val_loss: 0.4860 - val_accuracy: 0.7486
Epoch 4/5
104230/104230 [==============================] - 8s 77us/step - loss: 0.4791 - accuracy: 0.7602 - val_loss: 0.5004 - val_accuracy: 0.7380
Epoch 5/5
104230/104230 [==============================] - 8s 77us/step - loss: 0.4779 - accuracy: 0.7606 - val_loss: 0.4674 - val_accuracy: 0.7667


In [0]:
# 予測
predictions = model.predict(lastx)

In [225]:
print(predictions)

[[9.8589492e-01 1.3393462e-02]
 [9.2309368e-01 8.6373538e-02]
 [9.9963999e-01 3.0797720e-04]
 ...
 [4.5196521e-01 5.5536395e-01]
 [7.5320387e-01 2.7754349e-01]
 [7.7988619e-01 2.4828213e-01]]


In [226]:
# 評価
score = model.evaluate(lastx, lasty, verbose=1)


352595/352595 [==============================] - 13s 37us/step


In [227]:
print("Test score", score[0])
print("Test accuracy", score[1])

Test score 0.3927183591845654
Test accuracy 0.8215445280075073


In [0]:
# 回収率の計算

In [0]:
## 予想結果ともとのデータを結合

In [0]:
predict = pd.DataFrame(predictions, columns=["lose","win"],index=lastx.index)

In [0]:
result =  pd.merge(predict, lastsdat, right_index=True, left_index=True)

In [0]:
# リターンと支払額を計算

In [0]:
return_amount = result.query("win > 0.8")[["return"]].sum()["return"]
pay_amount =result.query("win > 0.8").count()["lose"]

In [287]:
# 最終損益
return_amount - pay_amount 

-147.89999999999998

In [291]:
# 回収率
return_amount / pay_amount 

0.707707509881423

In [0]:
# 的中率の計算

In [0]:
# 勝率が高いと予想し、かつ実際に3位以内に入った馬
success = result.query("win > 0.8 and rank <= 3").count()["lose"]

# 勝率が高いと予想した馬の合計数
total = result.query("win > 0.8").count()["lose"]

In [290]:
# 的中率
success / total

0.7885375494071146

In [0]:
# 競馬予想終わり

In [285]:
# 回収率を上げるための方策 start

0.0

In [0]:
 return_amount = result.query("win > 0.7 and odds >= 3").sum()["return"]
 pay_amount = result.query("win > 0.7 and odds >= 3").count()["return"]

In [321]:
return_amount/pay_amount

0.9519230769230768

In [309]:
sdat

,rank,frame_num,horse_num,name,sex,age,amount,jockey,time,diff_arrival,odds,popularity,weight,diff_weight,trainer,race_num,grade,ground,direction,unknown,distance,weather,condition,start_time,year,month,day,prev_rank,prev_frame_num,prev_horse_num,prev_name,prev_sex,prev_age,prev_amount,prev_jockey,prev_time,prev_diff_arrival,prev_odds,prev_popularity,prev_weight,prev_diff_weight,prev_trainer,prev_race_num,prev_grade,prev_ground,prev_direction,prev_unknown,prev_distance,prev_weather,prev_condition,prev_start_time,prev_year,prev_month,prev_day,winner,return,prev_time_float,start_time_float,prev_start_time_float,prev_high_rank,high_rank,prev_rank_level,rank_level,prev_popularity_level,popularity_level,prev_odds_level,opdds_level,prev_speed,prev_speed_level
1,17.0,1,2,1376,1,8,57.0,304,1:22.9,9,148.4,17.0,492.0,0.0,115,12,9,0,2,-,1400.0,3,0,16:15,2008,4,26,12.0,4.0,8.0,1376.0,1.0,8.0,57.0,57.0,1:12.6,9.0,89.0,14.0,492.0,-2.0,115.0,8.0,9.0,1.0,0.0,-,1200.0,0.0,0.0,13:40,2008.0,1.0,5.0,False,0.0,1561165956,1561220100,1561210800,0,0,3.0,4.0,4.0,4.0,22.0,37.0,7.686563e-07,8.0
4,6.0,2,3,21581,0,5,55.0,68,1:10.4,4,73.0,11.0,466.0,2.0,164,6,9,0,0,-,1200.0,0,0,12:30,2012,4,8,16.0,3.0,5.0,21581.0,0.0,5.0,55.0,334.0,1:40.1,7.0,66.3,13.0,464.0,-1.0,164.0,7.0,9.0,0.0,0.0,外,1600.0,3.0,3.0,13:20,2012.0,1.0,22.0,False,0.0,1561167606,1561206600,1561209600,0,0,4.0,2.0,3.0,3.0,17.0,18.0,1.024874e-06,10.0
6,12.0,5,7,24470,1,3,56.0,118,2:02.4,3,433.2,13.0,494.0,12.0,55,12,9,0,0,-,2000.0,0,0,16:10,2011,12,17,12.0,8.0,14.0,24470.0,1.0,3.0,55.0,26.0,2:30.7,9.0,202.8,13.0,482.0,-2.0,55.0,6.0,9.0,0.0,2.0,-,2400.0,3.0,0.0,12:30,2011.0,11.0,19.0,False,0.0,1561170642,1561219800,1561206600,0,0,3.0,3.0,3.0,3.0,51.0,108.0,1.537308e-06,15.0
8,14.0,6,11,11205,1,3,56.0,63,2:28.8,7,30.2,9.0,504.0,-6.0,69,11,2,0,2,-,2400.0,1,1,15:40,2008,5,3,4.0,8.0,9.0,11205.0,1.0,3.0,56.0,20.0,2:14.6,6.0,13.2,6.0,510.0,0.0,69.0,9.0,9.0,0.0,0.0,外,2200.0,0.0,0.0,14:30,2008.0,4.0,5.0,False,0.0,1561169676,1561218000,1561213800,0,0,1.0,4.0,2.0,2.0,3.0,8.0,1.409200e-06,14.0
14,1.0,1,1,9921,0,4,55.0,64,1:12.2,0,5.7,3.0,450.0,-4.0,39,12,9,1,0,-,1200.0,0,0,16:20,2009,3,28,8.0,2.0,4.0,9921.0,0.0,4.0,55.0,49.0,1:25.1,1.0,4.4,2.0,454.0,-4.0,39.0,8.0,9.0,1.0,0.0,-,1400.0,1.0,1.0,13:40,2009.0,2.0,22.0,True,5.7,1561166706,1561220400,1561210800,0,1,2.0,0.0,0.0,1.0,1.0,1.0,8.967652e-07,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542846,11.0,8,15,13001,0,4,55.0,184,1:10.0,2,213.1,15.0,436.0,-1.0,453,8,9,1,0,-,1150.0,1,1,14:01,2009,7,5,15.0,3.0,6.0,13001.0,0.0,3.0,51.0,249.0,1:49.6,17.0,23.7,8.0,430.0,14.0,24.0,1.0,9.0,1.0,2.0,-,1700.0,0.0,1.0,10:10,2008.0,6.0,7.0,False,0.0,1561168176,1561212060,1561198200,0,0,4.0,3.0,2.0,4.0,6.0,53.0,1.088928e-06,11.0
542848,4.0,8,15,35258,1,4,56.0,57,1:52.0,7,9.5,7.0,512.0,4.0,115,11,9,1,0,-,1800.0,0,2,15:35,2014,1,11,4.0,3.0,6.0,35258.0,1.0,3.0,56.0,83.0,1:53.5,9.0,7.2,4.0,508.0,-4.0,115.0,9.0,9.0,1.0,0.0,-,1800.0,0.0,2.0,14:35,2013.0,12.0,22.0,False,0.0,1561168410,1561217700,1561214100,0,0,1.0,1.0,1.0,2.0,2.0,2.0,1.152983e-06,12.0
542849,3.0,7,13,13702,0,3,54.0,75,1:50.3,1,4.6,2.0,458.0,-4.0,132,2,9,0,0,-,1800.0,0,0,10:20,2009,2,7,3.0,5.0,10.0,13702.0,0.0,3.0,54.0,75.0,1:52.3,14.0,3.6,2.0,462.0,-6.0,132.0,4.0,9.0,0.0,2.0,-,1800.0,0.0,0.0,11:30,2009.0,1.0,24.0,False,0.0,1561168338,1561198800,1561203000,1,1,1.0,1.0,0.0,0.0,1.0,1.0,1.152983e-06,12.0
542853,3.0,7,10,25151,1,7,58.0,194,1:46.1,3,15.8,5.0,482.0,-4.0,143,11,9,0,0,外,1800.0,1,1,15:35,2015,5,16,12.0,3.0,6.0,25151.0,1.0,7.0,56.0,79.0,1:33.6,4.0,8.6,3.0,486.0,0.0,143.0,11.0,3.0,0.0,0.0,外,1600.0,2.0,0.0,15:45,2015.0,4.0,5.0,False,0.0,1561167216,1561217700,1561218300,0,1,3.0,1.0,1.0,1.0,2.0,4.0,1.024874e-06,10.0


In [0]:
# 回収率を上げるための方策 end

In [253]:
predictions

array([[9.8589492e-01, 1.3393462e-02],
       [9.2309368e-01, 8.6373538e-02],
       [9.9963999e-01, 3.0797720e-04],
       ...,
       [4.5196521e-01, 5.5536395e-01],
       [7.5320387e-01, 2.7754349e-01],
       [7.7988619e-01, 2.4828213e-01]], dtype=float32)